In [ ]:
%pip install requests
%pip install azure-ai-vision-imageanalysis

In [3]:
import requests
from azure.ai.vision.imageanalysis import ImageAnalysisClient
from azure.ai.vision.imageanalysis.models import VisualFeatures
from azure.core.credentials import AzureKeyCredential

In [15]:
import requests
import gradio as gr
from PIL import Image, ImageDraw

def request_vision(image_path):
    features = "objects"
    endpoint = f"https://fimtrus-computervision2.cognitiveservices.azure.com/computervision/imageanalysis:analyze?api-version=2024-02-01&features={features}"
    api_key = "ee6d3d3de333480291b572fc7147b015"


    headers = {
        "Content-Type": "application/octet-stream",
        "Ocp-Apim-Subscription-Key": api_key
    }

    with open(image_path, 'rb') as image:
        image_data = image.read()

    response = requests.post(endpoint,
                            headers=headers,
                            data=image_data)

    response_json = response.json()
    print(response_json)

    return response_json

def change_image(image_path):

    if image_path is None:
        return None
    
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)
    
    # request_vision을 통해 데이터를 받아 온다.
    # 받은 데이터를 파싱해서 image에 그린다.

    response_json = request_vision(image_path)
    # 받은 데이터를 파싱해서 image에 그린다.  
    if "objectsResult" in response_json and "values" in response_json["objectsResult"]:  
        for obj in response_json["objectsResult"]["values"]:  
            box = obj["boundingBox"]  
            x, y, w, h = box["x"], box["y"], box["w"], box["h"]  
            print(x, y, w, h)
            draw.rectangle([(x, y), (x + w, y + h)], outline="red", width=2)  
            if "tags" in obj:  
                for tag in obj["tags"]:  
                    draw.text((x, y - 10), tag["name"], fill="red")  

    return image

with gr.Blocks() as demo:

    input_image = gr.Image(label="이미지 선택", type="filepath", width=800)
    output_image = gr.Image(label="출력 이미지", type="pil", width=800, interactive=False)

    input_image.change(fn=change_image, inputs=[input_image], outputs=[output_image])

demo.launch()

Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


{'modelVersion': '2023-10-01', 'metadata': {'width': 3672, 'height': 2654}, 'objectsResult': {'values': [{'boundingBox': {'x': 560, 'y': 458, 'w': 1057, 'h': 1007}, 'tags': [{'name': 'person', 'confidence': 0.771}]}, {'boundingBox': {'x': 2159, 'y': 769, 'w': 1004, 'h': 1026}, 'tags': [{'name': 'person', 'confidence': 0.829}]}]}}
560 458 1057 1007
2159 769 1004 1026
